![Piggy bank](piggy_bank.jpg)

Personal loans are a lucrative revenue stream for banks. The typical interest rate of a two-year loan in the United Kingdom is [around 10%](https://www.experian.com/blogs/ask-experian/whats-a-good-interest-rate-for-a-personal-loan/). This might not sound like a lot, but in September 2022 alone UK consumers borrowed [around £1.5 billion](https://www.ukfinance.org.uk/system/files/2022-12/Household%20Finance%20Review%202022%20Q3-%20Final.pdf), which would mean approximately £300 million in interest generated by banks over two years!

You have been asked to work with a bank to clean the data they collected as part of a recent marketing campaign, which aimed to get customers to take out a personal loan. They plan to conduct more marketing campaigns going forward so would like you to ensure it conforms to the specific structure and data types that they specify so that they can then use the cleaned data you provide to set up a PostgreSQL database, which will store this campaign's data and allow data from future campaigns to be easily imported. 

They have supplied you with a csv file called `"bank_marketing.csv"`, which you will need to clean, reformat, and split the data, saving three final csv files. Specifically, the three files should have the names and contents as outlined below:

## `client.csv`

| column | data type | description | cleaning requirements |
|--------|-----------|-------------|-----------------------|
| `client_id` | `integer` | Client ID | N/A |
| `age` | `integer` | Client's age in years | N/A |
| `job` | `object` | Client's type of job | Change `"."` to `"_"` |
| `marital` | `object` | Client's marital status | N/A |
| `education` | `object` | Client's level of education | Change `"."` to `"_"` and `"unknown"` to `np.NaN` |
| `credit_default` | `bool` | Whether the client's credit is in default | Convert to `boolean` data type:<br> `1` if `"yes"`, otherwise `0` |
| `mortgage` | `bool` | Whether the client has an existing mortgage (housing loan) | Convert to boolean data type:<br> `1` if `"yes"`, otherwise `0` |

<br>

## `campaign.csv`

| column | data type | description | cleaning requirements |
|--------|-----------|-------------|-----------------------|
| `client_id` | `integer` | Client ID | N/A |
| `number_contacts` | `integer` | Number of contact attempts to the client in the current campaign | N/A |
| `contact_duration` | `integer` | Last contact duration in seconds | N/A |
| `previous_campaign_contacts` | `integer` | Number of contact attempts to the client in the previous campaign | N/A |
| `previous_outcome` | `bool` | Outcome of the previous campaign | Convert to boolean data type:<br> `1` if `"success"`, otherwise `0`. |
| `campaign_outcome` | `bool` | Outcome of the current campaign | Convert to boolean data type:<br> `1` if `"yes"`, otherwise `0`. |
| `last_contact_date` | `datetime` | Last date the client was contacted | Create from a combination of `day`, `month`, and a newly created `year` column (which should have a value of `2022`); <br> **Format =** `"YYYY-MM-DD"` |

<br>

## `economics.csv`

| column | data type | description | cleaning requirements |
|--------|-----------|-------------|-----------------------|
| `client_id` | `integer` | Client ID | N/A |
| `cons_price_idx` | `float` | Consumer price index (monthly indicator) | N/A |
| `euribor_three_months` | `float` | Euro Interbank Offered Rate (euribor) three-month rate (daily indicator) | N/A |

In [43]:
import pandas as pd
import numpy as np

# Start coding here...

In [44]:
df = pd.read_csv("bank_marketing.csv")

for col in ["credit_default", "mortgage", "previous_outcome", "campaign_outcome"]:
    print(col)
    print("--------------")
    print(df[col].value_counts())

credit_default
--------------
no         32588
unknown     8597
yes            3
Name: credit_default, dtype: int64
mortgage
--------------
yes        21576
no         18622
unknown      990
Name: mortgage, dtype: int64
previous_outcome
--------------
nonexistent    35563
failure         4252
success         1373
Name: previous_outcome, dtype: int64
campaign_outcome
--------------
no     36548
yes     4640
Name: campaign_outcome, dtype: int64


## work on client.csv file

In [45]:
df_client= df[["client_id","age","job","marital","education","credit_default","mortgage"]]
df_client.head()

,client_id,age,job,marital,education,credit_default,mortgage
0,0,56,housemaid,married,basic.4y,no,no
1,1,57,services,married,high.school,unknown,no
2,2,37,services,married,high.school,no,yes
3,3,40,admin.,married,basic.6y,no,no
4,4,56,services,married,high.school,no,no


In [46]:
#
df_client["job"]=df_client.job.str.replace(".","_")
df_client.head()

,client_id,age,job,marital,education,credit_default,mortgage
0,0,56,housemaid,married,basic.4y,no,no
1,1,57,services,married,high.school,unknown,no
2,2,37,services,married,high.school,no,yes
3,3,40,admin_,married,basic.6y,no,no
4,4,56,services,married,high.school,no,no


In [47]:
#replacing "." with "_" and "unkonwn" to np.nan in education column
df_client["education"]=df_client.education.str.replace(".","_")
df_client["education"]= df_client.education.apply(lambda row: np.nan if row =="unknown" else row)

In [48]:
#replacing yes with 1 and coverting it to boolean datatype
df_client["credit_default"]= df_client.credit_default.apply(lambda row: 1 if row=="yes" else 0)
df_client["mortgage"]= df_client.mortgage.apply(lambda row: 1 if row=="yes" else 0)

In [49]:
df_client["credit_default"]=df_client.credit_default.astype("bool")
df_client["mortgage"]=df_client.mortgage.astype("bool")

In [50]:
df_client.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41188 entries, 0 to 41187
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   client_id       41188 non-null  int64 
 1   age             41188 non-null  int64 
 2   job             41188 non-null  object
 3   marital         41188 non-null  object
 4   education       39457 non-null  object
 5   credit_default  41188 non-null  bool  
 6   mortgage        41188 non-null  bool  
dtypes: bool(2), int64(2), object(3)
memory usage: 1.6+ MB


In [51]:
df_client.to_csv("client.csv",index=False)

## working on campaign.csv file

In [52]:
df_camp = df[["client_id","number_contacts","contact_duration","previous_campaign_contacts","previous_outcome","campaign_outcome"]]
df_camp.head()

,client_id,number_contacts,contact_duration,previous_campaign_contacts,previous_outcome,campaign_outcome
0,0,1,261,0,nonexistent,no
1,1,1,149,0,nonexistent,no
2,2,1,226,0,nonexistent,no
3,3,1,151,0,nonexistent,no
4,4,1,307,0,nonexistent,no


In [53]:
# converting to bool
df_camp["previous_outcome"] = df_camp["previous_outcome"].apply(lambda row: bool(1) if row=="success" else bool(0))
df_camp["campaign_outcome"] = df_camp["campaign_outcome"].apply(lambda row: bool(1) if row=="yes" else bool(0))
df_camp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41188 entries, 0 to 41187
Data columns (total 6 columns):
 #   Column                      Non-Null Count  Dtype
---  ------                      --------------  -----
 0   client_id                   41188 non-null  int64
 1   number_contacts             41188 non-null  int64
 2   contact_duration            41188 non-null  int64
 3   previous_campaign_contacts  41188 non-null  int64
 4   previous_outcome            41188 non-null  bool 
 5   campaign_outcome            41188 non-null  bool 
dtypes: bool(2), int64(4)
memory usage: 1.3 MB


In [54]:
# convert month column of df from monht name to month number
from calendar import month_abbr

lower_ma = [m.lower() for m in month_abbr]

df["month"] = df["month"].apply(lambda month : str(lower_ma.index(month)))
df["month"].head()

0    5
1    5
2    5
3    5
4    5
Name: month, dtype: object

In [55]:
# Create from a combination of day, month, and a newly created year column (which should have a value of 2022);
Format = "YYYY-MM-DD"
df_camp["last_contact_date"] =  "2022-"+df["month"]+"-"+ df["day"].astype("str")

In [56]:
df_camp.last_contact_date.head()

0    2022-5-13
1    2022-5-19
2    2022-5-23
3    2022-5-27
4     2022-5-3
Name: last_contact_date, dtype: object

In [57]:
# to save the csv file
df_camp.to_csv("campaign.csv",index=False)

## work on economics.csv file

In [58]:
df_eco = df[["client_id","cons_price_idx","euribor_three_months"]]
df_eco.to_csv("economics.csv",index=False)